## File for creating downsampled daily counts of EQs and their related parameters 

In [1]:
import pandas as pd
import dask.dataframe as dd
import hvplot.pandas  # noqa
#import hvplot.dask  # noqa
import numpy as np
from scipy import stats

In [2]:
!ls /home/jovyan/data/axial/axial_seismic/

axial_eq_daily.csv  hypo71_12062023.dat


In [3]:
!head /home/jovyan/data/axial/axial_seismic/hypo71_12062023.dat

yyyymmdd HHMM SSS.SS Lat(D M)  Lon(D M)  Depth    MW NWR GAP DMIN  RMS  ERH  ERZ            ID         PMom    SMom
20150122 0000 27.54 45 56.81 129 59.92   0.55  -0.31  7 206  1.2 0.03  0.8  0.4         20830      3.8e+17 3.9e+17
20150122 0008 58.89 45 56.96 129 59.70   0.00   0.02 10 149  1.2 0.09  0.2  0.2         20831      1.9e+18 7.8e+17
20150122 0202 41.04 45 56.47 130  0.89   0.67  -0.03 12 200  0.8 0.03  0.9  0.7         20836      1.4e+18 7.2e+17
20150122 0219 56.90 45 54.93 129 57.59   1.60   0.26  7 325  1.8 0.06  1.4  2.5         20837      3.3e+18 2.2e+18
20150122 0236  7.08 45 56.80 129 59.71   0.51  -0.15  8 160  1.2 0.06  0.5  0.2         20840      6.7e+17 6.8e+17
20150122 0239 20.66 45 56.93 129 59.48   1.56   0.07 12 148  0.9 0.08  0.5  1.3         20841      1.3e+18 1.6e+18
20150122 0242 34.06 45 56.70 129 59.34   1.09   0.27 12 108  0.9 0.05  0.3  1.4         20843      2.7e+18 3.1e+18
20150122 0250 15.94 45 57.96 129 59.53   1.45   0.54 14 286  1.8 0.03  0.7  1.1

In [4]:
seismic_file = '/home/jovyan/data/axial/axial_seismic/hypo71_12062023.dat'
df_seismic = pd.read_csv(seismic_file, delim_whitespace=True, dtype=object)
df_seismic['datetime'] = df_seismic['yyyymmdd'] + 'T' + \
             df_seismic['HHMM'].str.slice(start=0, stop=2) + ':' + \
             df_seismic['HHMM'].str.slice(start=2) 
df_seismic.index = pd.to_datetime(df_seismic['datetime'].values)
df_seismic['datetime'] = pd.to_datetime(df_seismic['datetime'].values)
#df_seismic = df_seismic.loc['2015-05-01 00:00:00':'2019-10-14 00:00:00']
df_seismic.datetime.astype(np.int64).values/1e64
df_seismic['date'] =pd.DatetimeIndex(df_seismic.datetime).date
df_seismic['Depth']= df_seismic['Depth'].astype(float).values
df_seismic['MW']= df_seismic['MW'].astype(float).values
df_seismic['RMS']= df_seismic['RMS'].astype(float).values
del df_seismic['NWR']
del df_seismic['GAP']
del df_seismic['DMIN']
del df_seismic['ERH']
del df_seismic['ERZ']
del df_seismic['ID']
del df_seismic['PMom']
del df_seismic['SMom']
del df_seismic['yyyymmdd']
del df_seismic['HHMM']
del df_seismic['SSS.SS']
del df_seismic['Lat(D']
del df_seismic['M)']
del df_seismic['Lon(D']
del df_seismic['M).1']
df_seismic.head()

,Depth,MW,RMS,datetime,date
2015-01-22 00:00:00,0.55,-0.31,0.03,2015-01-22 00:00:00,2015-01-22
2015-01-22 00:08:00,0.00,0.02,0.09,2015-01-22 00:08:00,2015-01-22
2015-01-22 02:02:00,0.67,-0.03,0.03,2015-01-22 02:02:00,2015-01-22
2015-01-22 02:19:00,1.60,0.26,0.06,2015-01-22 02:19:00,2015-01-22
2015-01-22 02:36:00,0.51,-0.15,0.06,2015-01-22 02:36:00,2015-01-22


## Create new dataframe with count per day

In [5]:
df_eqMean=df_seismic.groupby('date').mean()
df_eqCount= df_seismic.groupby('date').count()
df_eqCount['count'] = df_eqCount.datetime.astype('float64').values
df_eqMean['count'] = df_eqCount['count'].values
df_eqMean.head()

,Depth,MW,RMS,datetime,count
date,,,,,
2015-01-22,1.103731,0.058955,0.054030,2015-01-22 09:38:34.925373184,67.0
2015-01-23,0.925211,0.196761,0.028028,2015-01-23 09:54:21.126760704,71.0
2015-01-24,1.138940,0.157682,0.030199,2015-01-24 08:05:16.291390720,151.0
2015-01-25,1.056500,0.108900,0.027200,2015-01-25 15:57:59.400000000,100.0
2015-01-26,1.118314,0.149571,0.027686,2015-01-26 12:07:30.171428608,350.0


In [6]:
df_eqMean.to_csv('/home/jovyan/data/axial/axial_seismic/axial_eq_daily.csv') 

In [7]:
df_eqMean.hvplot.step(x='date', y=[ 'count'])

:Curve   [date]   (count)

In [11]:
seismic_file = '/home/jovyan/data/axial/axial_seismic/axial_eq_daily.csv'
df_test = pd.read_csv(seismic_file, dtype=object)
df_test.index = pd.to_datetime(df_test['date'].values)
df_test['datetime'] = pd.to_datetime(df_test['datetime'].values)
#df_seismic = df_seismic.loc['2015-05-01 00:00:00':'2019-10-14 00:00:00']
df_test.datetime.astype(np.int64).values/1e64
df_test['date'] =pd.DatetimeIndex(df_test.datetime).date
del df_test['date']

In [12]:
df_test.head()

,Depth,MW,RMS,datetime,count
2015-01-22,1.103731343283582,0.05895522388059702,0.054029850746268655,2015-01-22 09:38:34.925373184,67.0
2015-01-23,0.9252112676056338,0.19676056338028167,0.028028169014084506,2015-01-23 09:54:21.126760704,71.0
2015-01-24,1.1389403973509933,0.157682119205298,0.030198675496688743,2015-01-24 08:05:16.291390720,151.0
2015-01-25,1.0565,0.10890000000000001,0.027200000000000002,2015-01-25 15:57:59.400000000,100.0
2015-01-26,1.1183142857142858,0.14957142857142858,0.027685714285714286,2015-01-26 12:07:30.171428608,350.0
